In [ ]:
# ==============================
# 📦 Core Python Libraries
# ==============================
import json
from datetime import datetime

# ==============================
# 🧮 Scientific & Data Libraries
# ==============================
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt, welch

# ==============================
# 🫀 Signal Processing & EDF I/O
# ==============================
import pyedflib
from util.read_edf import read_edf_metadata, read_edf_to_dataframes

# ==============================
# 🧠 Quality Control Modules
# ==============================
from quality.ecg_quality import run_ecg_qc
from quality.flow_quality import run_flow_qc
from quality.technical_quality import run_clipping_qc

# ==============================
# 📊 Visualization
# ==============================
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.graph_objects as go
from plotly.subplots import make_subplots


### READ THE METADATA

In [ ]:
edf_metadata_df = read_edf_metadata("ABC100110013333PSG06.edf")
edf_metadata_df.head(50)

### READ THE CHANNEL INTO DATAFRAMES

In [ ]:
channel_dataframes = read_edf_to_dataframes("ABC100110013333PSG06.edf")
# The EDF file contains 23 signals (EOG, EEG, EMG, ECG, etc.).

In [ ]:
print(channel_dataframes.keys())

In [ ]:
ecg_ii_df = channel_dataframes['ECG II']
ecg_ii_df = ecg_ii_df.rename(columns={'Relative Time (s)': 'Time (s)', 'ECG II_Data': 'ECG II'})
ecg_ii_df["Absolute Time"] = pd.to_datetime(ecg_ii_df["Absolute Time"])

In [ ]:
channel_dataframes["ECG IIHF"]

### CHECK THE TECHNICAL QUALITY

In [ ]:
per_epoch, per_metric, overall = run_clipping_qc(
    channel_name="Technical (RI_Diag_Device Alice_5)",
    channel_dataframes=channel_dataframes,
    fs=200,
    epoch_len=30,
    rail_min=0.0,
    rail_max=65535.0,
    near_pct=0.01,
    plot=True,
    json_path=False
)
print(overall)

### CHECK THE ECG QUALITY

In [ ]:
import numpy as np
import pandas as pd
import neurokit2 as nk
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt

# --- Parameters ---
fs = 250  # sampling frequency
epoch_len_sec = 30
signal = ecg_ii_df["ECG II"].values.astype(float)

# --- Helper: Flatline detection ---
def flatline_ratio(sig, eps=1e-6):
    if len(sig) < 2:
        return 1.0
    diffs = np.abs(np.diff(sig))
    return float(np.mean(diffs < eps))

# --- Create augmented signal: add inverted first half to the end ---
half_len = len(signal) // 2
signal_aug = np.concatenate([signal, -1 * signal[:half_len]])

# --- Filter design ---
b, a = butter(4, (0.25, 25), 'bandpass', fs=fs)
epoch_len = fs * epoch_len_sec
n_epochs = len(signal_aug) // epoch_len

records = []
filtered_full = np.zeros_like(signal_aug)

for i in range(n_epochs):
    start = i * epoch_len
    end = start + epoch_len
    segment = signal_aug[start:end]

    try:
        ecg_filt = filtfilt(b, a, segment)
        ecg_clean = nk.ecg_clean(ecg_filt, sampling_rate=fs)
    except Exception:
        ecg_clean = segment

    filtered_full[start:end] = ecg_clean

    # --- Flatline check ---
    flat_ratio = flatline_ratio(ecg_clean)
    is_flat = flat_ratio > 0.95  # if >95% points unchanged → flatline

    if is_flat or np.std(ecg_clean) == 0:
        inv_ratio = 0  # mark as normal (non-inverted)
        was_inverted = False
    else:
        # Inversion metric
        r = np.corrcoef(ecg_clean, np.abs(ecg_clean))[0, 1]
        inv_ratio = (1 - r) / 2
        try:
            _, was_inverted = nk.ecg_invert(ecg_clean, sampling_rate=fs, show=False)
        except Exception:
            was_inverted = np.nan

    records.append({
        "epoch": i,
        "start_time_s": start / fs,
        "end_time_s": end / fs,
        "flatline_ratio": flat_ratio,
        "inversion_ratio": inv_ratio,
        "was_inverted": was_inverted,
        "is_flatline": is_flat
    })

df_epochs = pd.DataFrame(records)
df_epochs["high_inversion"] = (df_epochs["inversion_ratio"] > 0.5) & (~df_epochs["is_flatline"])

# --- Create time axis ---
t_all = np.arange(len(filtered_full)) / fs

# --- Matplotlib Figure ---
plt.figure(figsize=(15, 5))
plt.plot(t_all, filtered_full, color='black', linewidth=0.8, label="Filtered ECG")

# Highlight epochs: red = inverted, green = normal or flatline
for _, row in df_epochs.iterrows():
    color = "red" if row["high_inversion"] else "green"
    plt.axvspan(row["start_time_s"], row["end_time_s"], color=color, alpha=0.18)

# --- Layout customization ---
plt.title("ECG Inversion Detection — Flatline Treated as Normal", fontsize=14)
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (a.u.)")
plt.legend(loc="upper right")
plt.tight_layout()
plt.show()

In [ ]:
per_epoch, per_metric_json, overall_json = run_ecg_qc(
    "ECG II",
    channel_dataframes=channel_dataframes,   # your dict with ECG data
    fs=200,
    epoch_len=30,
    thresholds={
        "clipping_max": 0.50,
        "flatline_max": 0.50,
        "missing_max":  0.50,
        "baseline_max": 0.15,
        "hr_min": 25.0,
        "hr_max": 220.0,
        "snr_min": 5.0,
    },
    json_path="qc_summary.json",
    plot="per-metric"
)

print(overall_json)

In [ ]:

# --- Save per_epoch to a separate JSON file ---
with open("per_epoch_ecg.json", "w") as f:
    json.dump(per_epoch, f, indent=2)

print("✅ Saved per-epoch QC details to per_epoch.json")

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# ---------------- Parameters ----------------
json_path = "per_epoch_ecg.json"   # path to your saved file
time_format = "%Y-%m-%d %H:%M:%S"  # adjust if timestamps differ

# ---------------- Load JSON ----------------
with open(json_path, "r") as f:
    per_epoch = json.load(f)

df = pd.DataFrame(per_epoch)
print(f"✅ Loaded {len(df)} epochs from {json_path}")

# ---------------- Convert Times ----------------
df["Start_Time"] = pd.to_datetime(df["Start_Time"], errors="coerce")
df["End_Time"] = pd.to_datetime(df["End_Time"], errors="coerce")
df["Mid_Time"] = df["Start_Time"] + (df["End_Time"] - df["Start_Time"]) / 2

# ---------------- Available Metrics ----------------
metric_flags = {
    "Clipping_Ratio": "Bad_Clip",
    "Flatline_Ratio": "Bad_Flatline",
    "Missing_Ratio": "Bad_Missing",
    "Baseline_Wander_Ratio": "Bad_Baseline",
    "HR_Mean": "Bad_HR",
    "SNR_dB": "Bad_SNR",
}

available_metrics = [m for m in metric_flags if m in df.columns]
print(f"📊 Available metrics for plotting: {available_metrics}")

# ---------------- Per-Metric Plots ----------------
for metric in available_metrics:
    bad_flag = metric_flags[metric]
    if bad_flag not in df.columns:
        continue

    # Time vs Metric value
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(df["Mid_Time"], df[metric], "k.-", label=metric)

    # Shade good/bad epochs
    for _, row in df.iterrows():
        if pd.isna(row["Start_Time"]) or pd.isna(row["End_Time"]):
            continue
        color = "red" if row[bad_flag] else "green"
        ax.axvspan(row["Start_Time"], row["End_Time"], color=color, alpha=0.15)

    ax.set_title(f"{metric} — QC Highlight (Green = Good, Red = Bad)")
    ax.set_xlabel("Time")
    ax.set_ylabel(metric)
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
    ax.grid(True)
    plt.tight_layout()
    plt.show()

print("✅ Finished generating per-metric QC plots.")


# ---------------- Raw ECG Waveform Reconstruction ----------------
if "Raw_Data" in df.columns:
    print("🫀 Plotting continuous raw ECG waveform with QC shading...")

    # Combine all raw samples
    raw_all = np.concatenate([
        np.array(x, dtype=float) if isinstance(x, list) else np.array([])
        for x in df["Raw_Data"]
    ])

    # Approximate sampling rate
    total_duration = (df["End_Time"].iloc[-1] - df["Start_Time"].iloc[0]).total_seconds()
    fs = len(raw_all) / total_duration if total_duration > 0 else 200

    # Generate continuous timestamp array
    t_start = df["Start_Time"].iloc[0]
    t_all = pd.date_range(start=t_start, periods=len(raw_all), freq=pd.Timedelta(seconds=1/fs))

    print(f"Reconstructed ECG: {len(raw_all)} samples, fs ≈ {fs:.2f} Hz")

    # Plot continuous ECG with shaded epochs
    fig, ax = plt.subplots(figsize=(14, 5))
    ax.plot(t_all, raw_all, lw=0.7, color="black")

    # Shade epochs based on QC
    for _, row in df.iterrows():
        if pd.isna(row["Start_Time"]) or pd.isna(row["End_Time"]):
            continue
        color = "red" if row["Bad_Epoch"] else "green"
        ax.axvspan(row["Start_Time"], row["End_Time"], color=color, alpha=0.18)

    ax.set_title("ECG Raw Signal — QC Overlay (Green = Good, Red = Bad)")
    ax.set_xlabel("Time")
    ax.set_ylabel("Amplitude (a.u.)")
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
    ax.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No 'Raw_Data' field found in JSON — skipping raw signal plot.")


### READ FLOW Thermistor

In [ ]:
airflow_thermistor_df = channel_dataframes['Flow Patient (Thermistor)']

In [ ]:
airflow_thermistor_df = channel_dataframes['Flow Patient (Thermistor)']
airflow_thermistor_df.head()

In [ ]:
# NeuroKit2 for respiration rate (pip install neurokit2)
try:
    import neurokit2 as nk
    HAVE_NK = True
except Exception:
    HAVE_NK = False

# ---------------- Parameters ----------------
fs = 100                   # Hz
epoch_len = 30             # seconds (non-overlapping)
low_bpm_warn = 4.0         # draw a line at 4 BPM
bp_lo, bp_hi = 0.10, 1.00  # Hz bandpass for airflow (resp band)

# ---------------- Prepare Data ----------------
airflow_thermistor_df["Absolute Time"] = pd.to_datetime(
    airflow_thermistor_df["Absolute Time"], errors="coerce"
)
df_plot = airflow_thermistor_df.iloc[-120000:-10000].copy()

# keep valid rows and coerce numeric
df_plot = df_plot.dropna(subset=["Absolute Time", "Flow Patient (Thermistor)"]).copy()
df_plot["Flow"] = pd.to_numeric(df_plot["Flow Patient (Thermistor)"], errors="coerce")
df_plot = df_plot.dropna(subset=["Flow"])

times  = df_plot["Absolute Time"].to_numpy()
signal = df_plot["Flow"].to_numpy()

# optional light detrend/standardize to help filtering/nk
if len(signal):
    signal = signal - np.nanmedian(signal)

# ---------------- Bandpass filter (0.1–1.0 Hz) ----------------
def bandpass(signal, fs, lo, hi, order=4):
    nyq = 0.5 * fs
    lo_n = max(lo / nyq, 1e-6)
    hi_n = min(hi / nyq, 0.999999)
    b, a = butter(order, [lo_n, hi_n], btype="bandpass")
    return filtfilt(b, a, signal, method="gust")

sig_filt = bandpass(signal, fs, bp_lo, bp_hi)

# ---------------- Sliding-Window BPM on filtered signal ----------------
samples_per_epoch = int(fs * epoch_len)
step = samples_per_epoch  # no overlap
n = len(sig_filt)

bpm_times, bpm_values = [], []

def bpm_welch(seg, fs, band=(0.10, 1.00)):
    f, pxx = welch(seg, fs=fs, nperseg=min(len(seg), 2048))
    low, high = band
    mask = (f >= low) & (f <= high) & np.isfinite(pxx)
    if np.any(mask) and np.nansum(pxx[mask]) > 0:
        dom = f[mask][np.argmax(pxx[mask])]
        return float(dom * 60.0)
    return np.nan

for start in range(0, n - samples_per_epoch + 1, step):
    seg = sig_filt[start:start + samples_per_epoch]
    if len(seg) < samples_per_epoch:
        continue
    seg_time = times[start + samples_per_epoch // 2]

    bpm = np.nan
    if HAVE_NK:
        try:
            # Use NeuroKit2 on the filtered segment
            rr = nk.rsp_rate(seg, sampling_rate=fs, method="fft")
            if rr is not None and np.size(rr):
                bpm = float(np.nanmedian(rr))
            if not np.isfinite(bpm):
                rr2 = nk.rsp_rate(seg, sampling_rate=fs, method="count")
                if rr2 is not None and np.size(rr2):
                    bpm = float(np.nanmedian(rr2))
        except Exception:
            bpm = np.nan

    if not np.isfinite(bpm):
        bpm = bpm_welch(seg, fs, band=(bp_lo, bp_hi))

    bpm_times.append(seg_time)
    bpm_values.append(bpm)

bpm_df = pd.DataFrame({"Time": bpm_times, "BPM": bpm_values})

# ---------------- Plotly: raw + filtered + BPM ----------------
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True,
    row_heights=[0.65, 0.35], vertical_spacing=0.08,
    subplot_titles=(
        f"Airflow Thermistor (Raw vs. {bp_lo}-{bp_hi} Hz Bandpassed)",
        "Estimated Breathing Rate (BPM, per 30s window)"
    )
)

# Top: raw (thin) and filtered (thicker)
fig.add_trace(
    go.Scatter(x=times, y=signal, mode="lines",
               line=dict(width=1, color="rgba(30,144,255,0.6)"),
               name="Raw"),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=times, y=sig_filt, mode="lines",
               line=dict(width=2, color="black"),
               name=f"Filtered {bp_lo}-{bp_hi} Hz"),
    row=1, col=1
)

# Bottom: BPM time series
if len(bpm_df):
    fig.add_trace(
        go.Scatter(x=bpm_df["Time"], y=bpm_df["BPM"],
                   mode="lines+markers",
                   line=dict(width=2, color="crimson"),
                   name="BPM"),
        row=2, col=1
    )
    fig.add_hline(y=low_bpm_warn, line_dash="dot", line_color="gray", row=2, col=1)
else:
    fig.add_annotation(text="No BPM points (insufficient data / all-NaN).",
                       xref="paper", yref="paper", x=0.5, y=0.25,
                       showarrow=False, font=dict(color="gray"))

fig.update_layout(
    template="plotly_white",
    height=700, width=1000,
    legend=dict(orientation="h", y=1.08, x=1, xanchor="right")
)
fig.update_xaxes(title_text="Time", row=2, col=1)
fig.update_yaxes(title_text="Flow (Thermistor)", row=1, col=1)
fig.update_yaxes(title_text="Breaths/Minute", row=2, col=1, range=[0, 40])

fig.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.signal import butter, filtfilt, welch, correlate
import neurokit2 as nk
import json
from datetime import datetime, timezone, timedelta

# ---- helper metrics ----
def check_clipping(signal, digital_min=-100, digital_max=100, edge_pct=0.01):
    if signal.size == 0:
        return np.nan
    lower, upper = digital_min * (1 - edge_pct), digital_max * (1 - edge_pct)
    clipped = (signal <= lower) | (signal >= upper)
    return float(np.mean(clipped))

def flatline_ratio(signal, eps=1e-6):
    if signal.size < 2:
        return np.nan
    diffs = np.abs(np.diff(signal, prepend=signal[:1]))
    return float(np.mean(diffs < eps))

def missing_ratio(n_present, n_expected):
    if n_expected <= 0:
        return np.nan
    return float(max(0.0, 1.0 - n_present / n_expected))

def bandpass_filter(sig, fs, lo=0.10, hi=1.00, order=4):
    nyq = 0.5 * fs
    lo_n, hi_n = max(lo / nyq, 1e-6), min(hi / nyq, 0.999999)
    b, a = butter(order, [lo_n, hi_n], btype="bandpass")
    return filtfilt(b, a, sig, method="gust")

def bpm_welch(seg, fs, band=(0.10, 1.00)):
    if seg.size == 0:
        return np.nan
    f, pxx = welch(seg, fs=fs, nperseg=min(len(seg), 2048))
    m = (f >= band[0]) & (f <= band[1])
    if np.any(m) and np.nansum(pxx[m]) > 0:
        dom = f[m][np.argmax(pxx[m])]
        return float(dom * 60.0)
    return np.nan

# ---- autocorrelation quality ----
def autocorr_quality(seg, fs, max_lag_sec=10):
    """Compute normalized autocorrelation peak within lag window."""
    if len(seg) < fs:
        return np.nan
    seg = seg - np.nanmean(seg)
    ac = correlate(seg, seg, mode='full')
    ac = ac[len(ac)//2:]
    if np.nanmax(np.abs(ac)) == 0:
        return np.nan
    ac /= np.nanmax(ac)
    lags = np.arange(len(ac)) / fs
    mask = (lags >= 1.0) & (lags <= max_lag_sec)
    return float(np.nanmax(ac[mask])) if np.any(mask) else np.nan

def ratio_summary(bad_n, total):
    good_n = total - bad_n
    return {
        "good_epochs": int(good_n),
        "bad_epochs": int(bad_n),
        "good_ratio": round(good_n / total, 3) if total else None,
        "bad_ratio": round(bad_n / total, 3) if total else None,
    }

# ---- main QC ----
def run_flow_qc(
    channel_name,
    channel_dataframes,
    fs=100,
    epoch_len=30,
    json_path=None,
    plot="per-metric",
    clipping_max=0.50,
    flatline_max=0.50,
    missing_max=0.50,
    bpm_min=10.0,
    bpm_max=22.0,
    auto_min=0.5   # 🔸 threshold for autocorrelation quality
):
    if channel_name not in channel_dataframes:
        raise KeyError(f"Channel '{channel_name}' not found.")

    df = channel_dataframes[channel_name]
    t_abs = pd.to_datetime(df["Absolute Time"], errors="coerce")
    if getattr(t_abs.dt, "tz", None) is None:
        t_abs = t_abs.dt.tz_localize("UTC")

    t_abs_ns = t_abs.astype("int64", copy=False)
    sig_np = pd.to_numeric(df[channel_name], errors="coerce").to_numpy(dtype=float)
    mask = np.isfinite(sig_np) & np.isfinite(t_abs_ns)
    sig, t_abs_ns = sig_np[mask].astype(np.float32), t_abs_ns[mask]

    if sig.size == 0:
        empty = {
            "total_epochs": 0, "good_epochs": 0, "bad_epochs": 0,
            "good_ratio": None, "bad_ratio": None
        }
        if json_path:
            with open(json_path, "w") as f:
                json.dump({"per_epoch": [], "per_metric": {}, "overall": empty}, f, indent=2)
        return [], {}, empty

    # --- prepare time base ---
    t0_ns = t_abs_ns[0]
    t_sec = (t_abs_ns - t0_ns) / 1e9
    t0_dt = datetime.fromtimestamp(t0_ns / 1e9, tz=timezone.utc)

    spp = int(fs * epoch_len)
    starts = np.arange(0, len(sig), spp, dtype=int)
    ends = np.minimum(starts + spp, len(sig))

    per_epoch = []
    for i, (s, e) in enumerate(zip(starts, ends), start=1):
        seg = sig[s:e]
        clip = check_clipping(seg)
        flat = flatline_ratio(seg)
        miss = missing_ratio(seg.size, spp)

        try:
            seg_filt = bandpass_filter(seg - np.nanmedian(seg), fs)
        except Exception:
            seg_filt = seg

        bpm = np.nan
        try:
            rr = nk.rsp_rate(seg_filt, sampling_rate=fs, method="fft")
            if rr is not None and np.size(rr):
                bpm = float(np.nanmedian(rr))
            if not np.isfinite(bpm):
                bpm2 = nk.rsp_rate(seg_filt, sampling_rate=fs, method="count")
                if bpm2 is not None and np.size(bpm2):
                    bpm = float(np.nanmedian(bpm2))
        except Exception:
            pass
        if not np.isfinite(bpm):
            bpm = bpm_welch(seg_filt, fs)

        # --- Autocorrelation ---
        ac_score = autocorr_quality(seg_filt, fs)
        bad_auto = bool(np.isfinite(ac_score) and ac_score < auto_min)

        # --- QC flags ---
        bad_clip = bool(np.isfinite(clip) and clip > clipping_max)
        bad_flat = bool(np.isfinite(flat) and flat > flatline_max)
        bad_miss = bool(np.isfinite(miss) and miss > missing_max)
        bad_bpm_nan = bool(not np.isfinite(bpm))
        bad_bpm_low = bool(np.isfinite(bpm) and bpm < bpm_min)
        bad_bpm_high = bool(np.isfinite(bpm) and bpm > bpm_max)
        bad_bpm = bool(bad_bpm_low or bad_bpm_high or bad_bpm_nan)
        bad_epoch = bool(bad_clip or bad_flat or bad_miss or bad_bpm or bad_auto)

        per_epoch.append({
            "Epoch": int(i),
            "Start_Time_ISO": (t0_dt + timedelta(seconds=float(t_sec[s]))).isoformat(),
            "End_Time_ISO": (t0_dt + timedelta(seconds=float(t_sec[e - 1]))).isoformat(),
            "Clipping_Ratio": float(clip) if np.isfinite(clip) else None,
            "Flatline_Ratio": float(flat) if np.isfinite(flat) else None,
            "Missing_Ratio": float(miss) if np.isfinite(miss) else None,
            "BPM": float(bpm) if np.isfinite(bpm) else None,
            "AutoCorr_Q": float(ac_score) if np.isfinite(ac_score) else None,
            "Bad_Epoch": bad_epoch,
            "Bad_Clip": bad_clip,
            "Bad_Flatline": bad_flat,
            "Bad_Missing": bad_miss,
            "Bad_BPM": bad_bpm,
            "Bad_AutoCorr": bad_auto,
            "Raw_Data": seg.tolist()
        })

    # --- summaries ---
    total = len(per_epoch)
    def count(flag): return sum(1 for r in per_epoch if r[flag])
    per_metric_json = {
        "Clipping": ratio_summary(count("Bad_Clip"), total),
        "Flatline": ratio_summary(count("Bad_Flatline"), total),
        "Missing": ratio_summary(count("Bad_Missing"), total),
        "BPM": ratio_summary(count("Bad_BPM"), total),
        "AutoCorr_Q": ratio_summary(count("Bad_AutoCorr"), total)
    }
    overall_bad = count("Bad_Epoch")
    overall_json = {
        "total_epochs": total,
        "good_epochs": total - overall_bad,
        "bad_epochs": overall_bad,
        "good_ratio": round((total - overall_bad) / total, 3) if total else None,
        "bad_ratio": round(overall_bad / total, 3) if total else None,
    }

    # --- optional JSON save ---
    if json_path:
        with open(json_path, "w") as f:
            json.dump(
                {"per_epoch": per_epoch, "per_metric": per_metric_json, "overall": overall_json},
                f, indent=2
            )

    # --- plotting ---
    if plot in ("overall", "per-metric", "both"):
        times = [t0_dt + timedelta(seconds=float(s)) for s in t_sec]

        def shade(ax, flag_key):
            for r in per_epoch:
                st, et = datetime.fromisoformat(r["Start_Time_ISO"]), datetime.fromisoformat(r["End_Time_ISO"])
                ax.axvspan(st, et, color=("red" if r[flag_key] else "green"), alpha=0.18)

        if plot in ("overall", "both"):
            fig, ax = plt.subplots(figsize=(14, 5))
            step = max(1, len(sig) // 20000)
            ax.plot(times[::step], sig[::step], lw=0.8, color="black")
            shade(ax, "Bad_Epoch")
            ax.set_title(f"{channel_name} — Overall Flow QC (Red=Bad, Green=Good)")
            ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
            ax.grid(True); plt.tight_layout(); plt.show()

        if plot in ("per-metric", "both"):
            flag_map = {
                "Clipping": "Bad_Clip",
                "Flatline": "Bad_Flatline",
                "Missing": "Bad_Missing",
                "BPM": "Bad_BPM",
                "AutoCorr_Q": "Bad_AutoCorr"
            }
            for metric, flag in flag_map.items():
                fig, ax = plt.subplots(figsize=(14, 5))
                step = max(1, len(sig) // 20000)
                ax.plot(times[::step], sig[::step], lw=0.8, color="black")
                shade(ax, flag)
                ax.set_title(f"{channel_name} — {metric} QC (Red=Bad, Green=Good)")
                ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
                ax.grid(True); plt.tight_layout(); plt.show()

    return per_epoch, per_metric_json, overall_json


In [ ]:
per_epoch, per_metric_json, overall_json = run_flow_qc(
    channel_name="Flow Patient (Thermistor)",
    channel_dataframes=channel_dataframes,
    fs=100,
    epoch_len=30,
    json_path=False,          # or "qc_results.json" if you want to save to file
    plot="per-metric",        # options: "overall", "per-metric", "both", or 0 (none)
    clipping_max=0.50,
    flatline_max=0.50,
    missing_max=0.50,
    bpm_min=10.0,
    bpm_max=22.0,
    auto_min=0.5              # ✅ autocorrelation threshold (0–1 scale)
)

# Print overall QC summary
print(json.dumps(overall_json, indent=2))


In [ ]:
# --- Save per_epoch to a separate JSON file ---
with open("per_epoch_flow_temp.json", "w") as f:
    json.dump(per_epoch, f, indent=2)

print("✅ Saved per-epoch QC details to per_epoch.json")

In [ ]:
per_epoch

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# ---- Load the saved QC file ----
json_path = "per_epoch_flow_temp.json"

with open(json_path, "r") as f:
    per_epoch = json.load(f)

df = pd.DataFrame(per_epoch)
print(f"✅ Loaded {len(df)} epochs from {json_path}")

# ---- Convert times ----
df["Start_Time_ISO"] = pd.to_datetime(df["Start_Time_ISO"], errors="coerce")
df["End_Time_ISO"] = pd.to_datetime(df["End_Time_ISO"], errors="coerce")
df["Mid_Time"] = df["Start_Time_ISO"] + (df["End_Time_ISO"] - df["Start_Time_ISO"]) / 2

# ---- Define metrics and flags (AutoCorr_Q excluded from loop) ----
metric_flags = {
    "Clipping_Ratio": "Bad_Clip",
    "Flatline_Ratio": "Bad_Flatline",
    "Missing_Ratio": "Bad_Missing",
    "BPM": "Bad_BPM",
}

available_metrics = [m for m in metric_flags if m in df.columns]
print(f"📊 Available metrics for plotting: {available_metrics}")

# ---- Helper for shading ----
def shade_epochs(ax, start_col, end_col, flag_col):
    """Shade epochs red (bad) or green (good)."""
    for _, row in df.iterrows():
        if pd.isna(row[start_col]) or pd.isna(row[end_col]):
            continue
        color = "red" if row[flag_col] else "green"
        ax.axvspan(row[start_col], row[end_col], color=color, alpha=0.18)

# ---- Plot each QC metric ----
for metric in available_metrics:
    flag = metric_flags[metric]
    if flag not in df.columns:
        continue

    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(df["Mid_Time"], df[metric], "k.-", label=metric)

    # --- Add threshold lines ---
    if metric == "BPM":
        ax.axhline(10.0, color="gray", linestyle="--", lw=1, label="BPM Min (10)")
        ax.axhline(22.0, color="gray", linestyle="--", lw=1, label="BPM Max (22)")

    # --- Shading ---
    shade_epochs(ax, "Start_Time_ISO", "End_Time_ISO", flag)

    ax.set_title(f"{metric} — Flow QC (Green = Good, Red = Bad)")
    ax.set_xlabel("Time")
    ax.set_ylabel(metric)
    ax.legend()
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
    ax.grid(True)
    plt.tight_layout()
    plt.show()

# ---- Dedicated Autocorrelation Quality Plot ----
if "AutoCorr_Q" in df.columns:
    print("📈 Plotting dedicated Autocorrelation Quality (AutoCorr_Q)...")

    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(df["Mid_Time"], df["AutoCorr_Q"], "k.-", label="AutoCorr_Q")

    # Add autocorrelation threshold line
    ax.axhline(0.5, color="gray", linestyle="--", lw=1, label="AutoCorr Min (0.5)")

    shade_epochs(ax, "Start_Time_ISO", "End_Time_ISO", "Bad_AutoCorr")
    ax.set_title("Autocorrelation Quality (AutoCorr_Q) — Flow QC (Green = Good, Red = Bad)")
    ax.set_xlabel("Time")
    ax.set_ylabel("AutoCorr_Q (0–1)")
    ax.legend()
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
    ax.grid(True)
    plt.tight_layout()
    plt.show()

print("✅ Finished generating QC plots (AutoCorr_Q shown once).")

# ---- Combine all raw segments into one continuous trace ----
if "Raw_Data" in df.columns:
    print("🫀 Plotting raw waveform with QC shading...")

    # Concatenate all segments
    raw_all = np.concatenate([
        np.array(x, dtype=float) if isinstance(x, list) else np.array([])
        for x in df["Raw_Data"]
    ])

    # Approximate sampling rate and create time base
    total_duration = (df["End_Time_ISO"].iloc[-1] - df["Start_Time_ISO"].iloc[0]).total_seconds()
    fs = len(raw_all) / total_duration if total_duration > 0 else 100
    t_start = df["Start_Time_ISO"].iloc[0]
    t_all = pd.date_range(start=t_start, periods=len(raw_all), freq=pd.Timedelta(seconds=1/fs))

    print(f"Reconstructed raw signal: {len(raw_all)} samples, fs ≈ {fs:.2f} Hz")

    # ---- Plot raw waveform with shaded QC epochs ----
    fig, ax = plt.subplots(figsize=(14, 5))
    ax.plot(t_all, raw_all, lw=0.7, color="black")
    shade_epochs(ax, "Start_Time_ISO", "End_Time_ISO", "Bad_Epoch")
    ax.set_title("Flow Pressure Raw Signal — QC Overlay (Green = Good, Red = Bad)")
    ax.set_xlabel("Time")
    ax.set_ylabel("Amplitude")
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
    ax.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No 'Raw_Data' field found in JSON — skipping raw signal plot.")

### READ FLOW Pressure

In [ ]:
airflow_pressure_df = channel_dataframes['Flow Patient (Pressure)']

In [ ]:
# ---------------- Parameters ----------------
fs = 100                   # Hz
epoch_len = 30             # seconds (non-overlapping)
low_bpm_warn = 4.0         # draw a line at 4 BPM
bp_lo, bp_hi = 0.10, 1.00  # Hz bandpass for airflow (resp band)

# ---------------- Prepare Data ----------------
airflow_pressure_df ["Absolute Time"] = pd.to_datetime(
    airflow_pressure_df["Absolute Time"], errors="coerce"
)
df_plot = airflow_pressure_df.iloc[-120000:-10000].copy()

# keep valid rows and coerce numeric
df_plot = df_plot.dropna(subset=["Absolute Time", "Flow Patient (Pressure)"]).copy()
df_plot["Flow"] = pd.to_numeric(df_plot["Flow Patient (Pressure)"], errors="coerce")
df_plot = df_plot.dropna(subset=["Flow"])

times  = df_plot["Absolute Time"].to_numpy()
signal = df_plot["Flow"].to_numpy()

# optional light detrend/standardize to help filtering/nk
if len(signal):
    signal = signal - np.nanmedian(signal)

# ---------------- Bandpass filter (0.1–1.0 Hz) ----------------
def bandpass(signal, fs, lo, hi, order=4):
    nyq = 0.5 * fs
    lo_n = max(lo / nyq, 1e-6)
    hi_n = min(hi / nyq, 0.999999)
    b, a = butter(order, [lo_n, hi_n], btype="bandpass")
    return filtfilt(b, a, signal, method="gust")

sig_filt = bandpass(signal, fs, bp_lo, bp_hi)

# ---------------- Sliding-Window BPM on filtered signal ----------------
samples_per_epoch = int(fs * epoch_len)
step = samples_per_epoch  # no overlap
n = len(sig_filt)

bpm_times, bpm_values = [], []

def bpm_welch(seg, fs, band=(0.10, 1.00)):
    f, pxx = welch(seg, fs=fs, nperseg=min(len(seg), 2048))
    low, high = band
    mask = (f >= low) & (f <= high) & np.isfinite(pxx)
    if np.any(mask) and np.nansum(pxx[mask]) > 0:
        dom = f[mask][np.argmax(pxx[mask])]
        return float(dom * 60.0)
    return np.nan

for start in range(0, n - samples_per_epoch + 1, step):
    seg = sig_filt[start:start + samples_per_epoch]
    if len(seg) < samples_per_epoch:
        continue
    seg_time = times[start + samples_per_epoch // 2]

    bpm = np.nan
    if HAVE_NK:
        try:
            # Use NeuroKit2 on the filtered segment
            rr = nk.rsp_rate(seg, sampling_rate=fs, method="fft")
            if rr is not None and np.size(rr):
                bpm = float(np.nanmedian(rr))
            if not np.isfinite(bpm):
                rr2 = nk.rsp_rate(seg, sampling_rate=fs, method="count")
                if rr2 is not None and np.size(rr2):
                    bpm = float(np.nanmedian(rr2))
        except Exception:
            bpm = np.nan

    if not np.isfinite(bpm):
        bpm = bpm_welch(seg, fs, band=(bp_lo, bp_hi))

    bpm_times.append(seg_time)
    bpm_values.append(bpm)

bpm_df = pd.DataFrame({"Time": bpm_times, "BPM": bpm_values})

# ---------------- Plotly: raw + filtered + BPM ----------------
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True,
    row_heights=[0.65, 0.35], vertical_spacing=0.08,
    subplot_titles=(
        f"Airflow Thermistor (Raw vs. {bp_lo}-{bp_hi} Hz Bandpassed)",
        "Estimated Breathing Rate (BPM, per 30s window)"
    )
)

# Top: raw (thin) and filtered (thicker)
fig.add_trace(
    go.Scatter(x=times, y=signal, mode="lines",
               line=dict(width=1, color="rgba(30,144,255,0.6)"),
               name="Raw"),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=times, y=sig_filt, mode="lines",
               line=dict(width=2, color="black"),
               name=f"Filtered {bp_lo}-{bp_hi} Hz"),
    row=1, col=1
)

# Bottom: BPM time series
if len(bpm_df):
    fig.add_trace(
        go.Scatter(x=bpm_df["Time"], y=bpm_df["BPM"],
                   mode="lines+markers",
                   line=dict(width=2, color="crimson"),
                   name="BPM"),
        row=2, col=1
    )
    fig.add_hline(y=low_bpm_warn, line_dash="dot", line_color="gray", row=2, col=1)
else:
    fig.add_annotation(text="No BPM points (insufficient data / all-NaN).",
                       xref="paper", yref="paper", x=0.5, y=0.25,
                       showarrow=False, font=dict(color="gray"))

fig.update_layout(
    template="plotly_white",
    height=700, width=1000,
    legend=dict(orientation="h", y=1.08, x=1, xanchor="right")
)
fig.update_xaxes(title_text="Time", row=2, col=1)
fig.update_yaxes(title_text="Flow (Thermistor)", row=1, col=1)
fig.update_yaxes(title_text="Breaths/Minute", row=2, col=1, range=[0, 40])

fig.show()

In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt, welch, correlate
from datetime import datetime
import neurokit2 as nk
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# ---------------- Parameters ----------------
fs = 100                   # Sampling frequency (Hz)
epoch_len = 30             # Window length (seconds)
shift_sec = 5              # Shift between windows (seconds)
low_bpm_warn = 4.0         # Threshold line for BPM
bp_lo, bp_hi = 0.10, 1.00  # Bandpass limits for respiration (Hz)
HAVE_NK = True             # NeuroKit availability

# ---------------- Prepare Data ----------------
airflow_pressure_df["Absolute Time"] = pd.to_datetime(
    airflow_pressure_df["Absolute Time"], errors="coerce"
)
df_plot = airflow_pressure_df.iloc[-200000:-10000].copy()
df_plot = df_plot.dropna(subset=["Absolute Time", "Flow Patient (Pressure)"]).copy()
df_plot["Flow"] = pd.to_numeric(df_plot["Flow Patient (Pressure)"], errors="coerce")
df_plot = df_plot.dropna(subset=["Flow"])

times = df_plot["Absolute Time"].to_numpy()
signal = df_plot["Flow"].to_numpy()
if len(signal):
    signal = signal - np.nanmedian(signal)

# ---------------- Bandpass filter ----------------
def bandpass(signal, fs, lo, hi, order=4):
    nyq = 0.5 * fs
    lo_n, hi_n = max(lo / nyq, 1e-6), min(hi / nyq, 0.999999)
    b, a = butter(order, [lo_n, hi_n], btype="bandpass")
    return filtfilt(b, a, signal, method="gust")

sig_filt = bandpass(signal, fs, bp_lo, bp_hi)

# ---------------- Autocorrelation Quality ----------------
def autocorr_quality(seg, fs, max_lag_sec=10):
    """Compute normalized autocorrelation peak within a lag window."""
    if len(seg) < fs:
        return np.nan
    seg = seg - np.nanmean(seg)
    ac = correlate(seg, seg, mode='full')
    ac = ac[len(ac)//2:]  # keep positive lags
    if np.nanmax(np.abs(ac)) == 0:
        return np.nan
    ac /= np.nanmax(ac)
    lags = np.arange(len(ac)) / fs
    mask = (lags >= 1.0) & (lags <= max_lag_sec)
    return float(np.nanmax(ac[mask])) if np.any(mask) else np.nan

# ---------------- BPM Estimation ----------------
def bpm_welch(seg, fs, band=(0.10, 1.00)):
    """Estimate dominant frequency → BPM using Welch PSD."""
    f, pxx = welch(seg, fs=fs, nperseg=min(len(seg), 2048))
    mask = (f >= band[0]) & (f <= band[1])
    if np.any(mask) and np.nansum(pxx[mask]) > 0:
        dom = f[mask][np.argmax(pxx[mask])]
        return float(dom * 60.0)
    return np.nan

# ---------------- Sliding Window Analysis ----------------
samples_per_epoch = int(fs * epoch_len)
step = int(fs * shift_sec)
n = len(sig_filt)
bpm_times, bpm_values, ac_scores = [], [], []

for start in range(0, n - samples_per_epoch + 1, step):
    seg = sig_filt[start:start + samples_per_epoch]
    if len(seg) < samples_per_epoch:
        continue
    seg_time = times[start + samples_per_epoch // 2]

    # --- Estimate Breathing Rate (BPM) ---
    bpm = np.nan
    if HAVE_NK:
        try:
            rr = nk.rsp_rate(seg, sampling_rate=fs, method="fft")
            if rr is not None and np.size(rr):
                bpm = float(np.nanmedian(rr))
            if not np.isfinite(bpm):
                rr2 = nk.rsp_rate(seg, sampling_rate=fs, method="count")
                if rr2 is not None and np.size(rr2):
                    bpm = float(np.nanmedian(rr2))
        except Exception:
            bpm = np.nan

    if not np.isfinite(bpm):
        bpm = bpm_welch(seg, fs)

    # --- Autocorrelation Quality ---
    ac_score = autocorr_quality(seg, fs)

    bpm_times.append(seg_time)
    bpm_values.append(bpm)
    ac_scores.append(ac_score)

bpm_df = pd.DataFrame({
    "Time": bpm_times,
    "BPM": bpm_values,
    "AutoCorr_Q": ac_scores
})

# ---------------- Plotly Visualization ----------------
fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True,
    row_heights=[0.6, 0.25, 0.15],
    vertical_spacing=0.07,
    subplot_titles=(
        f"Airflow (Raw vs. {bp_lo}-{bp_hi} Hz Bandpassed)",
        f"Estimated Breathing Rate (BPM, {epoch_len}s windows, {shift_sec}s shift)",
        "Autocorrelation-Based Signal Quality"
    )
)

# Top: raw + filtered
fig.add_trace(go.Scatter(
    x=times, y=signal, mode="lines",
    line=dict(width=1, color="rgba(30,144,255,0.6)"),
    name="Raw"), 1, 1)

fig.add_trace(go.Scatter(
    x=times, y=sig_filt, mode="lines",
    line=dict(width=2, color="black"),
    name=f"Filtered {bp_lo}-{bp_hi} Hz"), 1, 1)

# BPM
fig.add_trace(go.Scatter(
    x=bpm_df["Time"], y=bpm_df["BPM"],
    mode="lines+markers",
    line=dict(width=2, color="crimson"),
    name="BPM"), 2, 1)
fig.add_hline(y=low_bpm_warn, line_dash="dot", line_color="gray", row=2, col=1)

# Autocorr quality
fig.add_trace(go.Scatter(
    x=bpm_df["Time"], y=bpm_df["AutoCorr_Q"],
    mode="lines+markers",
    line=dict(width=2, color="orange"),
    name="AutoCorr_Q"), 3, 1)

# Layout
fig.update_layout(
    template="plotly_white",
    height=900, width=1100,
    legend=dict(orientation="h", y=1.12, x=1, xanchor="right")
)
fig.update_xaxes(title_text="Time", row=3, col=1)
fig.update_yaxes(title_text="Flow", row=1, col=1)
fig.update_yaxes(title_text="BPM", range=[0, 40], row=2, col=1)
fig.update_yaxes(title_text="Autocorr (0–1)", range=[0, 1], row=3, col=1)

fig.show()

In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt, welch, correlate
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# ---------------- Parameters ----------------
fs = 100               # Hz
duration = 300         # 5 minutes
t = np.arange(0, duration, 1/fs)
freq_base = 0.2        # 12 BPM = 0.2 Hz base respiration
rng = np.random.default_rng(42)  # reproducibility

# ---------------- Base Clean Signal ----------------
signal_clean = np.sin(2 * np.pi * freq_base * t)
signal_noisy = signal_clean.copy()

# ---------------- Noise Section Definitions ----------------
# 0–60 s: low drift + light Gaussian noise
drift1 = 0.15 * np.sin(2 * np.pi * 0.02 * t[:fs*60])
signal_noisy[:fs*60] += drift1 + 0.03 * rng.standard_normal(fs*60)

# 60–120 s: in-band irregular breathing (0.3 Hz and 0.5 Hz modulation)
mix1 = (0.15 * np.sin(2 * np.pi * 0.3 * t[fs*60:fs*120]) +
        0.08 * np.sin(2 * np.pi * 0.5 * t[fs*60:fs*120]))
signal_noisy[fs*60:fs*120] += mix1 + 0.05 * rng.standard_normal(fs*60)

# 120–180 s: stronger in-band + motion noise (0.7–1 Hz)
mix2 = (0.20 * np.sin(2 * np.pi * 0.7 * t[fs*120:fs*180]) +
        0.12 * np.sin(2 * np.pi * 1.0 * t[fs*120:fs*180]))
signal_noisy[fs*120:fs*180] += mix2 + 0.08 * rng.standard_normal(fs*60)

# 180–210 s: high-frequency jitter (3–5 Hz)
hf1 = 0.08 * np.sin(2 * np.pi * 4.0 * t[fs*180:fs*210]) + 0.05 * rng.standard_normal(fs*30)
signal_noisy[fs*180:fs*210] += hf1

# 210–240 s: mixed HF jitter + low-frequency drift (combined artifacts)
hf2 = 0.06 * np.sin(2 * np.pi * 3.5 * t[fs*210:fs*240])
drift2 = 0.10 * np.sin(2 * np.pi * 0.05 * t[fs*210:fs*240])
signal_noisy[fs*210:fs*240] += hf2 + drift2 + 0.07 * rng.standard_normal(fs*30)

# 240–260 s: flatline (sensor dropout)
signal_noisy[fs*240:fs*260] = 0.0

# 260–300 s: recovery — mild drift + small in-band wobble + light noise
drift3 = 0.08 * np.sin(2 * np.pi * 0.04 * t[fs*260:])
rebound = 0.05 * np.sin(2 * np.pi * 0.25 * t[fs*260:])
signal_noisy[fs*260:] += drift3 + rebound + 0.02 * rng.standard_normal(len(t) - fs*260)

# ---------------- Bandpass Filter (0.1–1 Hz) ----------------
def bandpass(sig, fs, lo=0.1, hi=1.0, order=4):
    nyq = 0.5 * fs
    b, a = butter(order, [lo/nyq, hi/nyq], btype="bandpass")
    return filtfilt(b, a, sig, method="gust")

sig_filt = bandpass(signal_noisy, fs)

# ---------------- Autocorrelation Quality ----------------
def autocorr_quality(seg, fs, max_lag_sec=10):
    if len(seg) < fs:
        return np.nan
    seg = seg - np.nanmean(seg)
    ac = correlate(seg, seg, mode="full")[len(seg)-1:]
    ac /= np.nanmax(np.abs(ac)) if np.nanmax(np.abs(ac)) != 0 else 1
    lags = np.arange(len(ac)) / fs
    mask = (lags >= 1.0) & (lags <= max_lag_sec)
    return float(np.nanmax(ac[mask])) if np.any(mask) else np.nan

# ---------------- BPM Estimation ----------------
def bpm_welch(seg, fs, band=(0.1, 1.0)):
    f, pxx = welch(seg, fs=fs, nperseg=min(len(seg), 2048))
    m = (f >= band[0]) & (f <= band[1])
    if np.any(m) and np.nansum(pxx[m]) > 0:
        return float(f[m][np.argmax(pxx[m])] * 60.0)
    return np.nan

# ---------------- Sliding Windows (30 s, shift 5 s) ----------------
epoch_len, shift_sec = 30, 5
samples_per_epoch = int(fs * epoch_len)
step = int(fs * shift_sec)
n = len(sig_filt)

bpm_times, bpm_values, ac_scores = [], [], []
times = pd.date_range("2022-01-01", periods=len(t), freq=f"{1000/fs}ms")

for start in range(0, n - samples_per_epoch + 1, step):
    seg = sig_filt[start:start+samples_per_epoch]
    seg_time = times[start + samples_per_epoch//2]
    bpm = bpm_welch(seg, fs)
    ac = autocorr_quality(seg, fs)
    bpm_times.append(seg_time)
    bpm_values.append(bpm)
    ac_scores.append(ac)

bpm_df = pd.DataFrame({"Time": bpm_times, "BPM": bpm_values, "AutoCorr_Q": ac_scores})

# ---------------- Visualization ----------------
fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True,
    row_heights=[0.55, 0.25, 0.2], vertical_spacing=0.06,
    subplot_titles=(
        "Synthetic Respiration (12 BPM Base, Multi-Zone Noise & Dropout)",
        "Estimated Breathing Rate (BPM)",
        "Autocorrelation-Based Signal Quality"
    )
)

fig.add_trace(go.Scatter(x=times, y=signal_noisy, mode="lines",
                         line=dict(width=1, color="rgba(30,144,255,0.6)"),
                         name="Raw (variable noise)"), 1, 1)
fig.add_trace(go.Scatter(x=times, y=sig_filt, mode="lines",
                         line=dict(width=2, color="black"),
                         name="Filtered 0.1–1 Hz"), 1, 1)
fig.add_trace(go.Scatter(x=bpm_df["Time"], y=bpm_df["BPM"],
                         mode="lines+markers",
                         line=dict(width=2, color="crimson"),
                         name="BPM (Welch)"), 2, 1)
fig.add_hline(y=12, line_dash="dot", line_color="gray", row=2, col=1)
fig.add_trace(go.Scatter(x=bpm_df["Time"], y=bpm_df["AutoCorr_Q"],
                         mode="lines+markers",
                         line=dict(width=2, color="orange"),
                         name="Autocorr Q"), 3, 1)

fig.update_layout(
    template="plotly_white", height=950, width=1150,
    legend=dict(orientation="h", y=1.1, x=1, xanchor="right")
)
fig.update_xaxes(title_text="Time", row=3, col=1)
fig.update_yaxes(title_text="Flow", row=1, col=1)
fig.update_yaxes(title_text="BPM", range=[0, 25], row=2, col=1)
fig.update_yaxes(title_text="Autocorr (0–1)", range=[0, 1], row=3, col=1)
fig.show()


In [ ]:
per_epoch, per_metric_json, overall_json = run_flow_qc(
    channel_name="Flow Patient (Pressure)",
    channel_dataframes=channel_dataframes,
    fs=100,
    epoch_len=30,
    json_path=False,          # or "qc_results.json" if you want to save to file
    plot="per-metric",        # options: "overall", "per-metric", "both", or 0 (none)
    clipping_max=0.50,
    flatline_max=0.50,
    missing_max=0.50,
    bpm_min=10.0,
    bpm_max=22.0,
    auto_min=0.5              # ✅ autocorrelation threshold (0–1 scale)
)

# Print overall QC summary
print(json.dumps(overall_json, indent=2))

In [ ]:
import json
# --- Save per_epoch to a separate JSON file ---
with open("per_epoch_flow_pressure.json", "w") as f:
    json.dump(per_epoch, f, indent=2)

print("✅ Saved per-epoch QC details to per_epoch.json")

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# ---- Load the saved QC file ----
json_path = "per_epoch_flow_pressure.json"

with open(json_path, "r") as f:
    per_epoch = json.load(f)

df = pd.DataFrame(per_epoch)
print(f"✅ Loaded {len(df)} epochs from {json_path}")

# ---- Convert times ----
df["Start_Time_ISO"] = pd.to_datetime(df["Start_Time_ISO"], errors="coerce")
df["End_Time_ISO"] = pd.to_datetime(df["End_Time_ISO"], errors="coerce")
df["Mid_Time"] = df["Start_Time_ISO"] + (df["End_Time_ISO"] - df["Start_Time_ISO"]) / 2

# ---- Define metrics and flags (AutoCorr_Q excluded from loop) ----
metric_flags = {
    "Clipping_Ratio": "Bad_Clip",
    "Flatline_Ratio": "Bad_Flatline",
    "Missing_Ratio": "Bad_Missing",
    "BPM": "Bad_BPM",
}

available_metrics = [m for m in metric_flags if m in df.columns]
print(f"📊 Available metrics for plotting: {available_metrics}")

# ---- Helper for shading ----
def shade_epochs(ax, start_col, end_col, flag_col):
    """Shade epochs red (bad) or green (good)."""
    for _, row in df.iterrows():
        if pd.isna(row[start_col]) or pd.isna(row[end_col]):
            continue
        color = "red" if row[flag_col] else "green"
        ax.axvspan(row[start_col], row[end_col], color=color, alpha=0.18)

# ---- Plot each QC metric ----
for metric in available_metrics:
    flag = metric_flags[metric]
    if flag not in df.columns:
        continue

    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(df["Mid_Time"], df[metric], "k.-", label=metric)

    # --- Add threshold lines ---
    if metric == "BPM":
        ax.axhline(10.0, color="gray", linestyle="--", lw=1, label="BPM Min (10)")
        ax.axhline(22.0, color="gray", linestyle="--", lw=1, label="BPM Max (22)")

    # --- Shading ---
    shade_epochs(ax, "Start_Time_ISO", "End_Time_ISO", flag)

    ax.set_title(f"{metric} — Flow QC (Green = Good, Red = Bad)")
    ax.set_xlabel("Time")
    ax.set_ylabel(metric)
    ax.legend()
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
    ax.grid(True)
    plt.tight_layout()
    plt.show()

# ---- Dedicated Autocorrelation Quality Plot ----
if "AutoCorr_Q" in df.columns:
    print("📈 Plotting dedicated Autocorrelation Quality (AutoCorr_Q)...")

    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(df["Mid_Time"], df["AutoCorr_Q"], "k.-", label="AutoCorr_Q")

    # Add autocorrelation threshold line
    ax.axhline(0.5, color="gray", linestyle="--", lw=1, label="AutoCorr Min (0.5)")

    shade_epochs(ax, "Start_Time_ISO", "End_Time_ISO", "Bad_AutoCorr")
    ax.set_title("Autocorrelation Quality (AutoCorr_Q) — Flow QC (Green = Good, Red = Bad)")
    ax.set_xlabel("Time")
    ax.set_ylabel("AutoCorr_Q (0–1)")
    ax.legend()
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
    ax.grid(True)
    plt.tight_layout()
    plt.show()

print("✅ Finished generating QC plots (AutoCorr_Q shown once).")

# ---- Combine all raw segments into one continuous trace ----
if "Raw_Data" in df.columns:
    print("🫀 Plotting raw waveform with QC shading...")

    # Concatenate all segments
    raw_all = np.concatenate([
        np.array(x, dtype=float) if isinstance(x, list) else np.array([])
        for x in df["Raw_Data"]
    ])

    # Approximate sampling rate and create time base
    total_duration = (df["End_Time_ISO"].iloc[-1] - df["Start_Time_ISO"].iloc[0]).total_seconds()
    fs = len(raw_all) / total_duration if total_duration > 0 else 100
    t_start = df["Start_Time_ISO"].iloc[0]
    t_all = pd.date_range(start=t_start, periods=len(raw_all), freq=pd.Timedelta(seconds=1/fs))

    print(f"Reconstructed raw signal: {len(raw_all)} samples, fs ≈ {fs:.2f} Hz")

    # ---- Plot raw waveform with shaded QC epochs ----
    fig, ax = plt.subplots(figsize=(14, 5))
    ax.plot(t_all, raw_all, lw=0.7, color="black")
    shade_epochs(ax, "Start_Time_ISO", "End_Time_ISO", "Bad_Epoch")
    ax.set_title("Flow Pressure Raw Signal — QC Overlay (Green = Good, Red = Bad)")
    ax.set_xlabel("Time")
    ax.set_ylabel("Amplitude")
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
    ax.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No 'Raw_Data' field found in JSON — skipping raw signal plot.")
